In [1]:
!pip install -q timm onnxruntime onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 89.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 14.1 MB/s eta 0:00:00


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import random

In [3]:
# ---------------------------
# Reproducibility
# ---------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [20]:
# ---------------------------
# Kaggle Dataset Paths
# ---------------------------
TRAIN_REAL   = os.path.join("/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/train/real")
TRAIN_ATTACK = os.path.join("/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/train/attack")
TEST_REAL    = os.path.join("/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/test/real")
TEST_ATTACK  = os.path.join("/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/test/attack")

In [31]:
# ---------------------------
# Dataset
# ---------------------------
class AntispoofDataset(Dataset):
    def __init__(self, real_path, attack_path, transform=None):
        self.samples = []
        self.transform = transform

        # Real = 1
        for identity in os.listdir(real_path):
            identity_path = os.path.join(real_path, identity)
            if os.path.isdir(identity_path):
                for img in os.listdir(identity_path):
                    if img.lower().endswith((".jpg",".jpeg",".png")):
                        self.samples.append((os.path.join(identity_path, img), 1))

        # Attack = 0
        for identity in os.listdir(attack_path):
            identity_path = os.path.join(attack_path, identity)
            if os.path.isdir(identity_path):
                for img in os.listdir(identity_path):
                    if img.lower().endswith((".jpg",".jpeg",".png")):
                        self.samples.append((os.path.join(identity_path, img), 0))

        real_count = sum(1 for _, label in self.samples if label == 1)
        attack_count = len(self.samples) - real_count
        print(f"Loaded {len(self.samples)} frames  - \nReal: {real_count} | Attack: {attack_count}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

In [32]:
# ---------------------------
# Transforms - HEAVY augmentation for small dataset
# ---------------------------
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

In [33]:
train_dataset = AntispoofDataset(TRAIN_REAL, TRAIN_ATTACK, train_transform)
test_dataset  = AntispoofDataset(TEST_REAL, TEST_ATTACK, test_transform)

Loaded 2806 frames  - 
Real: 1333 | Attack: 1473
Loaded 1302 frames  - 
Real: 637 | Attack: 665


In [34]:
# INCREASED BATCH SIZE: 32 → 64 for more stable gradients
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)

In [35]:
# ---------------------------
# Building Blocks
# ---------------------------
class DepthwiseSeparableConv(nn.Module):
    """
    Depthwise Separable Convolution (MobileNet-style)
    Reduces parameters while maintaining expressiveness
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels,kernel_size=3, stride=stride, padding=1,groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [36]:
class ResidualBlock(nn.Module):
    """
    Residual Block with Depthwise Separable Convs
    Skip connection for better gradient flow
    """
    def __init__(self, channels):
        super().__init__()
        self.conv1 = DepthwiseSeparableConv(channels, channels, stride=1)
        self.conv2 = nn.Sequential(nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False),nn.BatchNorm2d(channels))
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        out += residual  # Skip connection
        out = self.relu(out)
        return out


In [37]:
# ---------------------------
# Optimized Custom CNN Model
# ---------------------------
class OptimizedAntispoofCNN(nn.Module):
    """
    Optimized Custom CNN for Antispoofing
    
    KEY IMPROVEMENTS:
    1. Deeper backbone: 7 blocks (vs 5)
    2. Depthwise separable convolutions (fewer params)
    3. Residual connections (better gradient flow)
    4. 2 specialized branches: Texture + Color only
    5. Reduced regularization (dropout 0.2 vs 0.4)
    
    Architecture designed for small dataset (4K samples)
    """
    def __init__(self):
        super().__init__()
        
        # ============================================
        # STEM: Initial feature extraction
        # ============================================
        # Input: 224x224x3 → 112x112x32
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        
        # ============================================
        # BACKBONE: 7-Block Feature Extractor
        # ============================================
        
        # Block 1: 112x112x32 → 112x112x32 (with residual)
        self.block1 = nn.Sequential(
            ResidualBlock(32),
            ResidualBlock(32)
        )
        
        # Block 2: 112x112x32 → 56x56x64
        self.block2 = nn.Sequential(
            DepthwiseSeparableConv(32, 64, stride=2),
            ResidualBlock(64)
        )  # → TEXTURE BRANCH INPUT
        
        # Block 3: 56x56x64 → 28x28x128
        self.block3 = nn.Sequential(
            DepthwiseSeparableConv(64, 128, stride=2),
            ResidualBlock(128)
        )
        
        # Block 4: 28x28x128 → 28x28x128 (with residual)
        self.block4 = nn.Sequential(
            ResidualBlock(128),
            ResidualBlock(128)
        )  # → COLOR BRANCH INPUT
        
        # Block 5: 28x28x128 → 14x14x256
        self.block5 = nn.Sequential(
            DepthwiseSeparableConv(128, 256, stride=2),
            ResidualBlock(256)
        )
        
        # Block 6: 14x14x256 → 7x7x512
        self.block6 = nn.Sequential(
            DepthwiseSeparableConv(256, 512, stride=2),
            ResidualBlock(512)
        )
        
        # Block 7: 7x7x512 → 7x7x512 (with residual)
        self.block7 = nn.Sequential(
            ResidualBlock(512),
            ResidualBlock(512)
        )
        
        # ============================================
        # BRANCH 1: Texture Analysis
        # ============================================
        # Captures high-frequency artifacts (print dots, screen pixels)
        # Input: 56x56x64 from block2
        self.texture_branch = nn.Sequential(
            # Enhance texture details
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # Global texture descriptor
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)  # REDUCED from 0.3
        )  # Output: 64-dim
        
        # ============================================
        # BRANCH 2: Color Statistics Analysis
        # ============================================
        # Analyzes color distribution (screens have different gamut)
        # Input: 28x28x128 from block4
        self.color_branch = nn.Sequential(
            # Spatial color distribution (4x4 grid)
            nn.Conv2d(128, 64, kernel_size=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(4),  # 4x4 spatial grid
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),  # REDUCED from 0.3
            nn.Linear(128, 128),
            nn.ReLU(inplace=True)
        )  # Output: 128-dim
        
        # ============================================
        # MAIN BRANCH: Deep semantic features
        # ============================================
        self.main_pool = nn.AdaptiveAvgPool2d(1)
        
        # ============================================
        # FUSION & CLASSIFICATION
        # ============================================
        # Concatenate: Texture(64) + Color(128) + Main(512) = 704
        self.classifier = nn.Sequential(
            nn.Linear(64 + 128 + 512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),  # REDUCED from 0.4
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),  # REDUCED from 0.4
            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        # Stem
        x = self.stem(x)  # 112x112x32
        
        # Backbone
        x = self.block1(x)  # 112x112x32
        texture_feat = self.block2(x)  # 56x56x64 → Texture Branch
        x = self.block3(texture_feat)  # 28x28x128
        color_feat = self.block4(x)  # 28x28x128 → Color Branch
        x = self.block5(color_feat)  # 14x14x256
        x = self.block6(x)  # 7x7x512
        main_feat = self.block7(x)  # 7x7x512
        
        # Branch processing
        texture_out = self.texture_branch(texture_feat)  # 64-dim
        color_out = self.color_branch(color_feat)  # 128-dim
        main_out = self.main_pool(main_feat).flatten(1)  # 512-dim
        
        # Concatenate all features
        combined = torch.cat([texture_out, color_out, main_out], dim=1)  # 704-dim
        
        # Classification
        output = self.classifier(combined)
        
        return output

In [38]:
# ---------------------------
# Model Summary
# ---------------------------
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [39]:
model = OptimizedAntispoofCNN().to(device)
print(f"\n{'='*60}")
print(f"Optimized Custom Antispoof CNN")
print(f"{'='*60}")
print(f"Architecture: 7 blocks + 2 branches (Texture + Color)")
print(f"Total Trainable Parameters: {count_parameters(model):,}")
print(f"{'='*60}\n")


Optimized Custom Antispoof CNN
Architecture: 7 blocks + 2 branches (Texture + Color)
Total Trainable Parameters: 9,731,521



In [40]:
# ---------------------------
# Training Setup with Warmup
# ---------------------------
criterion = nn.BCEWithLogitsLoss()

# INCREASED LR: 1e-3 → 2e-3
optimizer = optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)  # REDUCED weight decay

# Cosine annealing with warmup
NUM_EPOCHS = 35  # INCREASED from 25
WARMUP_EPOCHS = 5

In [41]:
# Main scheduler (after warmup)
main_scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=NUM_EPOCHS - WARMUP_EPOCHS, 
    eta_min=1e-6
)

In [42]:
# Warmup scheduler
def warmup_lr(epoch):
    if epoch < WARMUP_EPOCHS:
        return (epoch + 1) / WARMUP_EPOCHS
    return 1.0

In [43]:
warmup_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=warmup_lr)

best_acc = 0
best_auc = 0


In [44]:
# ---------------------------
# Training Loop
# ---------------------------
print("Starting Training with Warmup...\n")

for epoch in range(NUM_EPOCHS):
    # ===== TRAINING PHASE =====
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        # Update progress bar
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    train_acc = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    # ===== VALIDATION PHASE =====
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = 100 * correct / total
    avg_val_loss = val_loss / len(test_loader)
    auc = roc_auc_score(all_labels, all_preds)

    # Learning rate step
    if epoch < WARMUP_EPOCHS:
        warmup_scheduler.step()
    else:
        main_scheduler.step()
    
    current_lr = optimizer.param_groups[0]['lr']

    # ===== PRINT RESULTS =====
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss:   {avg_val_loss:.4f} | Val Acc:   {test_acc:.2f}%")
    print(f"AUC Score:  {auc:.4f}")
    print(f"LR: {current_lr:.6f} {'[Warmup]' if epoch < WARMUP_EPOCHS else '[Main]'}")

    # Save best model
    if test_acc > best_acc or (test_acc == best_acc and auc > best_auc):
        best_acc = test_acc
        best_auc = auc
        torch.save(model.state_dict(), "/kaggle/working/best_optimized_cnn.pth")
        print(f"✓ Best model saved! (Acc: {best_acc:.2f}%, AUC: {best_auc:.4f})")

    print(f"{'='*60}\n")

print(f"\n{'='*60}")
print(f"Training Complete!")
print(f"Best Validation Accuracy: {best_acc:.2f}%")
print(f"Best AUC Score: {best_auc:.4f}")
print(f"{'='*60}\n")

Starting Training with Warmup...



Epoch 1/35 [Val]: 100%|██████████| 21/21 [00:03<00:00,  6.40it/s]



Epoch 1/35
Train Loss: 0.6296 | Train Acc: 66.00%
Val Loss:   0.5554 | Val Acc:   76.88%
AUC Score:  0.8174
LR: 0.000800 [Warmup]
✓ Best model saved! (Acc: 76.88%, AUC: 0.8174)



Epoch 2/35 [Val]: 100%|██████████| 21/21 [00:02<00:00,  8.66it/s]



Epoch 2/35
Train Loss: 0.5986 | Train Acc: 67.71%
Val Loss:   0.7172 | Val Acc:   61.60%
AUC Score:  0.7209
LR: 0.001200 [Warmup]



...


Epoch 33/35
Train Loss: 0.1863 | Train Acc: 92.05%
Val Loss:   0.4332 | Val Acc:   83.72%
AUC Score:  0.9221
LR: 0.000023 [Main]



Epoch 34/35 [Val]: 100%|██████████| 21/21 [00:02<00:00,  7.92it/s]



Epoch 34/35
Train Loss: 0.1961 | Train Acc: 91.87%
Val Loss:   0.4175 | Val Acc:   83.95%
AUC Score:  0.9262
LR: 0.000006 [Main]



Epoch 35/35 [Val]: 100%|██████████| 21/21 [00:02<00:00,  7.80it/s]


Epoch 35/35
Train Loss: 0.1839 | Train Acc: 91.87%
Val Loss:   0.4078 | Val Acc:   84.64%
AUC Score:  0.9261
LR: 0.000001 [Main]


Training Complete!
Best Validation Accuracy: 85.25%
Best AUC Score: 0.9291



In [45]:
# ---------------------------
# ONNX Export
# ---------------------------
print("Exporting model to ONNX format...")

onnx_path = "/kaggle/working/optimized_antispoof_cnn.onnx"

Exporting model to ONNX format...


In [46]:
# Load best model
model.load_state_dict(torch.load("/kaggle/working/best_optimized_cnn.pth"))
model.eval()

dummy_input = torch.randn(1, 3, 224, 224).to(device)

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    }
)

print(f"✓ ONNX model exported to: {onnx_path}")

/tmp/ipykernel_55/2426272916.py:7: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


✓ ONNX model exported to: /kaggle/working/optimized_antispoof_cnn.onnx


In [47]:
# ---------------------------
# Final Evaluation
# ---------------------------
print("\nRunning final evaluation on test set...")

model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images).squeeze()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()

        correct += (preds == labels).sum().item()
        total += labels.size(0)

        all_preds.extend(probs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

final_acc = 100 * correct / total
final_auc = roc_auc_score(all_labels, all_preds)

print(f"\n{'='*60}")
print(f"FINAL TEST RESULTS")
print(f"{'='*60}")
print(f"Accuracy: {final_acc:.2f}%")
print(f"AUC Score: {final_auc:.4f}")
print(f"{'='*60}\n")


Running final evaluation on test set...


100%|██████████| 21/21 [00:03<00:00,  6.91it/s]


FINAL TEST RESULTS
Accuracy: 85.25%
AUC Score: 0.9291

